In [ ]:
!pip install --upgrade spacy

In [ ]:
!python -m spacy download en_core_web_trf


In [ ]:
import os
import pandas as pd

In [ ]:
with open("/content/CDR_TrainingSet.PubTator.txt", "r") as trainfile:
    trainfile = trainfile.read()

with open("/content/CDR_TestSet.PubTator.txt", "r") as testfile:
    testfile = testfile.read()



In [ ]:
#train set
ctr = 0
train = []
for line in trainfile.split("\n\n"):
  train.append(line)



#test set
ctr = 0
test = []
for line in testfile.split("\n\n"):
  test.append(line)

In [ ]:
len(train)

501

In [ ]:
len(test)

501

extract articles

In [ ]:
import re
# Corrected get_title function
def get_title(text):
    title = re.findall(r"t\|(.*)\n", text)
    if title:
        return title[0]
    else:
        return "No title found"  # Return a default value if no title is found

def get_article(text):
    article = re.findall(r"a\|(.*)\n", text)
    if article:
        return article[0]
    else:
        return "No article found"



In [ ]:
#articls for train data
train_article = [get_title(x)+" "+ get_article(x) for x in train]
len(train_article)

#articles for test data
test_article = [get_title(x)+" "+ get_article(x) for x in test]
len(test_article)

501

adding articles to a dataframe

In [ ]:
train_df = pd.DataFrame(columns = ["article"])
train_df["article"] = train_article
train_df.head()

,article
0,Naloxone reverses the antihypertensive effect ...
1,Lidocaine-induced cardiac asystole. Intravenou...
2,Suxamethonium infusion rate and observed fasci...
3,"Galanthamine hydrobromide, a longer acting ant..."
4,Effects of uninephrectomy and high protein fee...


In [ ]:
test_df = pd.DataFrame(columns = ["article"])
test_df["article"] = test_article
test_df.head()

,article
0,Famotidine-associated delirium. A series of si...
1,Indomethacin induced hypotension in sodium and...
2,Late-onset scleroderma renal crisis induced by...
3,The risk and associated factors of methampheta...
4,Cerebellar sensory processing alterations impa...


In [ ]:
train_df['article'][0]

'Naloxone reverses the antihypertensive effect of clonidine. In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) to 10(-5) M, did not influence stereoselective binding of [3H]-naloxone (8 nM), and naloxone, 10(-8) to 10(-4) M, did not influence clonidine-suppressible binding of [3H]-dihydroergocryptine (1 nM). These findings indicate that in spontaneously hypertensive rats the effects of central alpha-adrenoceptor stimulation involve activation of opiate receptors. As naloxone and clonidine do not appear to interact with the same receptor site, the observed functional antagonism suggests the release 

get labels

In [ ]:


def get_labels(text):
    # Extract the lines with tab-separated values
    l = re.findall(r"\t(.*)", text)
    l = [x.split("\t") for x in l]
    labels = []
    for i in l:
        # Skip lines with 'CID' or lines that don't have enough elements
        if len(i) < 4 or i[0] == "CID":
            continue
        labels.append((int(i[0]), int(i[1]), i[3]))
    return labels

def get_labels_and_entity(text):
    # Extract the lines with tab-separated values
    l = re.findall(r"\t(.*)", text)
    l = [x.split("\t") for x in l]
    labels = []
    for i in l:
        # Skip lines with 'CID' or lines that don't have enough elements
        if len(i) < 4 or i[0] == "CID":
            continue
        labels.append((int(i[0]), int(i[1]), i[3], i[2]))  # Add the entity as well
    return labels


In [ ]:
train_labels = [get_labels(x) for x in train]
len(train_labels)

501

In [ ]:
test_labels = [get_labels(x) for x in test]
len(test_labels)

501

In [ ]:
train_df["labels"][0]

[(0, 8, 'Chemical'),
 (49, 58, 'Chemical'),
 (93, 105, 'Disease'),
 (181, 190, 'Chemical'),
 (244, 252, 'Chemical'),
 (274, 285, 'Disease'),
 (306, 322, 'Chemical'),
 (354, 362, 'Chemical'),
 (364, 372, 'Chemical'),
 (469, 481, 'Disease'),
 (487, 496, 'Chemical'),
 (563, 576, 'Chemical'),
 (589, 597, 'Chemical'),
 (637, 646, 'Chemical'),
 (671, 695, 'Chemical'),
 (750, 762, 'Disease'),
 (865, 873, 'Chemical'),
 (878, 887, 'Chemical'),
 (1026, 1035, 'Chemical'),
 (1039, 1055, 'Chemical')]

adding labels to the dataframe

In [ ]:
train_df["labels"] = train_labels
train_df.head()

,article,labels
0,Naloxone reverses the antihypertensive effect ...,"[(0, 8, Chemical), (49, 58, Chemical), (93, 10..."
1,Lidocaine-induced cardiac asystole. Intravenou...,"[(0, 9, Chemical), (18, 34, Disease), (90, 99,..."
2,Suxamethonium infusion rate and observed fasci...,"[(0, 13, Chemical), (41, 55, Disease), (80, 10..."
3,"Galanthamine hydrobromide, a longer acting ant...","[(0, 25, Chemical), (111, 122, Chemical), (124..."
4,Effects of uninephrectomy and high protein fee...,"[(54, 61, Chemical), (70, 91, Disease), (111, ..."


In [ ]:
test_df["labels"] = test_labels
test_df.head()

,article,labels
0,Famotidine-associated delirium. A series of si...,"[(0, 10, Chemical), (22, 30, Disease), (55, 65..."
1,Indomethacin induced hypotension in sodium and...,"[(0, 12, Chemical), (21, 32, Disease), (36, 42..."
2,Late-onset scleroderma renal crisis induced by...,"[(11, 35, Disease), (47, 57, Chemical), (62, 7..."
3,The risk and associated factors of methampheta...,"[(35, 50, Chemical), (51, 60, Disease), (64, 7..."
4,Cerebellar sensory processing alterations impa...,"[(78, 97, Disease), (110, 120, Disease), (192,..."


custom NER

prepare data

In [ ]:
 training_data = []
 for i, j in zip(train_article, train_labels):
  training_data.append((i, j))

In [ ]:
training_data[0]

('Naloxone reverses the antihypertensive effect of clonidine. In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) to 10(-5) M, did not influence stereoselective binding of [3H]-naloxone (8 nM), and naloxone, 10(-8) to 10(-4) M, did not influence clonidine-suppressible binding of [3H]-dihydroergocryptine (1 nM). These findings indicate that in spontaneously hypertensive rats the effects of central alpha-adrenoceptor stimulation involve activation of opiate receptors. As naloxone and clonidine do not appear to interact with the same receptor site, the observed functional antagonism suggests the release

In [ ]:
testing_data = []
for i, j in zip(test_article, test_labels):
  testing_data.append((i, j))

In [ ]:
testing_data[0]

('Famotidine-associated delirium. A series of six cases. Famotidine is a histamine H2-receptor antagonist used in inpatient settings for prevention of stress ulcers and is showing increasing popularity because of its low cost. Although all of the currently available H2-receptor antagonists have shown the propensity to cause delirium, only two previously reported cases have been associated with famotidine. The authors report on six cases of famotidine-associated delirium in hospitalized patients who cleared completely upon removal of famotidine. The pharmacokinetics of famotidine are reviewed, with no change in its metabolism in the elderly population seen. The implications of using famotidine in elderly persons are discussed.',
 [(0, 10, 'Chemical'),
  (22, 30, 'Disease'),
  (55, 65, 'Chemical'),
  (156, 162, 'Disease'),
  (324, 332, 'Disease'),
  (395, 405, 'Chemical'),
  (442, 452, 'Chemical'),
  (464, 472, 'Disease'),
  (537, 547, 'Chemical'),
  (573, 583, 'Chemical'),
  (689, 699, 

In [ ]:
!pip install spacy-transformers

In [ ]:
import spacy.training
import spacy_transformers
from spacy.tokens import DocBin

nlp = spacy.load('en_core_web_trf')

/usr/local/lib/python3.11/dist-packages/thinc/shims/pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, map_location=dev

In [ ]:
#for training data
db = DocBin()
for text, annotations in training_data:
    doc = nlp.make_doc(text)  # Faster than nlp(text) because it skips NER
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        if span is not None:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("train.spacy")


In [ ]:
#testing data
db = DocBin()
for text, annotations in testing_data:
    doc = nlp.make_doc(text)  # Faster than nlp(text) because it skips NER
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        if span is not None:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("dev.spacy")

train model

In [ ]:
!python -m spacy init fill-config --help

                                                                                                    
 Usage: python -m spacy init fill-config [OPTIONS] BASE_PATH [OUTPUT_FILE]                          
                                                                                                    
 Fill partial config file with default values. Will add all missing settings from the default       
 config and will create all objects, check the registered functions for their default values and    
 update the base config. This command can be used with a config generated via the training          
 quickstart widget: https://spacy.io/usage/training#quickstart                                      
 DOCS: https://spacy.io/api/cli#init-fill-config                                                    
                                                                                                    
╭─ Arguments ──────────────────────────────────────────────────────────────────────────────

In [ ]:
nlp = spacy.blank("en")  # Create an empty pipeline
doc_bin = DocBin().from_disk("train.spacy")
docs = list(doc_bin.get_docs(nlp.vocab))

# Print first doc to check
print(docs[0].text)           # Original text
print([(ent.text, ent.label_) for ent in docs[0].ents])  # Entities

Naloxone reverses the antihypertensive effect of clonidine. In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) to 10(-5) M, did not influence stereoselective binding of [3H]-naloxone (8 nM), and naloxone, 10(-8) to 10(-4) M, did not influence clonidine-suppressible binding of [3H]-dihydroergocryptine (1 nM). These findings indicate that in spontaneously hypertensive rats the effects of central alpha-adrenoceptor stimulation involve activation of opiate receptors. As naloxone and clonidine do not appear to interact with the same receptor site, the observed functional antagonism suggests the release o

In [ ]:
!python -m spacy init config base_config.cfg --lang en --pipeline ner --optimize efficiency



ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
base_config.cfg
You can now add your data and train your pipeline:
python -m spacy train base_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg


⚠ Nothing to auto-fill: base config is already complete
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


model training

In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy --gpu-id -1


ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     63.90    1.16    0.75    2.56    0.01
  0     200       1056.62   6765.74   50.85   66.12   41.31    0.51
  0     400        218.46   3486.94   67.18   74.89   60.91    0.67
  1     600        197.61   2480.25   71.02   75.45   67.08    0.71
  1     800        220.91   1779.79   74.53   79.76   69.94    0.75
  2    1000        227.15   1889.29   73.77   75.31   72.29    0.74
  2    1200        221.28   1286.13   75.07   80.34   70.44    0.75
  2    1400        208.97   1053.81   76.07   82.88   70.29    0.76
  3    1600        252.34   105

model testing

In [ ]:
unique_labels = set()

for example in testing_data:
  entities = example[1]
  for entity in entities:
    entity_label = entity[2]
    unique_labels.add(entity_label)

unique_labels_list = list(unique_labels)

colors = {
    "Chemical": "#85C1E9",
    "Disease": "#FF6347",
}

options = {"ents": ["Chemical", "Disease"], "colors": colors}

print("Entities to be recognised in the provided medical text : ")
print(unique_labels_list)

Entities to be recognised in the provided medical text : 
['Disease', 'Chemical']


In [ ]:
nlp = spacy.load("/content/output/model-best")

In [ ]:
doc = nlp(testing_data[5][0])
spacy.displacy.render(doc, style="ent", options=options, jupyter=True)